# Creating Azure Resources



In [ ]:
from azure.mgmt.resource import ResourceManagementClient
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource.resources.models import DeploymentMode
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
import json

Start with specifying your subscription and resource group information and log in.

In [ ]:
# Subscription & resource group
SUBSCRIPTION_NAME = "Boston Team Danielle"
RESOURCE_GROUP = "baimm"
LOCATION = "eastus"

In [ ]:
# Login to subscription 
!az login --use-device-code

In [ ]:
# Select subscription
!az account set -s "{SUBSCRIPTION_NAME}"

In [ ]:
# Get susbcription info
temp = '"az account show -s \\"{}\\""'.format(SUBSCRIPTION_NAME)
subscription_id, tenant_id  = !eval {temp} | jq -r '.id, .tenantId'

In [ ]:
# Create resource group
!az group create -l {LOCATION} -n {RESOURCE_GROUP}

The following are parameters needed to create and access the main Azure resources. These include: Azure Container Registry (ACR), Azure Machine Learning (AML), Blob Storage, and Logic Apps parameters. 

You can use the default values below as is.

In [ ]:
# AML
AML_WORKSPACE = "amlws"
AML_COMPUTE_NAME = "{}cluster".format(RESOURCE_GROUP)
AML_VM_SIZE = "Standard_D2"
AML_MIN_NODES = 2
AML_MAX_NODES = 2

# Scoring Script
PIP_PACKAGES = ["numpy", "scipy", "scikit-learn", "pandas"]
PYTHON_VERSION = "3.6.7"
PYTHON_SCRIPT_NAME = "predict.py"
PYTHON_SCRIPT_DIRECTORY = "."

# Blob storage
BLOB_ACCOUNT = "{}storage".format(RESOURCE_GROUP)
MODELS_CONTAINER = "models"
PREDS_CONTAINER = "preds"
DATA_CONTAINER = "data"
DATA_BLOB = "sensor_data.csv"  # name of data file to be copied to blob storage

# ACR
ACR_NAME = '{}acr'.format(RESOURCE_GROUP)
ACR_SERVER = '{}.azurecr.io'.format(ACR_NAME)
SCHED_DOCKER_IMG = '{}/schedimg:v1'.format(ACR_SERVER)

# Logic App
LA_ACI_CON = "aci"
LA_WORKFLOW = "{}scheduler".format(RESOURCE_GROUP)
LA_ACI_CON_JSON = "sched/api_con_template.json"
LA_JSON = "sched/logic_app_template.json"
LA_ACI_CONTAINER_NAME = "schedcontainer"
LA_ACI_CONTAINER_GROUP = "containergroup"
LA_ACI_COMMAND = ["python", "submit_jobs.py", "bai_pred_config.json"]


In [ ]:
# Create Blob storage account
!az storage account create -n {BLOB_ACCOUNT} -g {RESOURCE_GROUP} -l {LOCATION}

In [ ]:
# Retrieve Blob storage key
blob_key = !az storage account keys list -g {RESOURCE_GROUP} -n {BLOB_ACCOUNT} | jq -r .[0].value
blob_key = blob_key[0]

In [ ]:
# Create models, predictions and data containers
!az storage container create -n {MODELS_CONTAINER} --account-key {blob_key} --account-name {BLOB_ACCOUNT}
!az storage container create -n {PREDS_CONTAINER} --account-key {blob_key} --account-name {BLOB_ACCOUNT}
!az storage container create -n {DATA_CONTAINER} --account-key {blob_key} --account-name {BLOB_ACCOUNT}

The following commands copy the pre-trained models and sample data from this repo to blob storage so that AML can access them during job submission.

In [ ]:
# Copy models from local dir to blob container
!az storage blob upload-batch -d {MODELS_CONTAINER} -s models --account-name {BLOB_ACCOUNT} --account-key {blob_key}
# Copy dataset to blob
!az storage blob upload -c {DATA_CONTAINER} -f data/'{DATA_BLOB}' -n '{DATA_BLOB}' --account-name {BLOB_ACCOUNT} --account-key {blob_key}

In [ ]:
# Create AML Workspace
aml_ws = Workspace.create(
    name=AML_WORKSPACE,
    subscription_id=subscription_id,
    resource_group=RESOURCE_GROUP,
    create_resource_group=False,
    location=LOCATION,
)

In [ ]:
# Create compute target
provisioning_config = AmlCompute.provisioning_configuration(vm_size = AML_VM_SIZE,
                                                            min_nodes = AML_MIN_NODES,
                                                            max_nodes = AML_MAX_NODES)

compute_target = ComputeTarget.create(aml_ws, AML_COMPUTE_NAME, provisioning_config)
compute_target.wait_for_completion(show_output=True)

You will also need service principal credentials for authentication. The following command creates and retrieves the credentials. For more information on service principal, check the docuemntation [here](https://docs.microsoft.com/en-us/cli/azure/create-an-azure-service-principal-azure-cli?view=azure-cli-latest).

In [ ]:
# Create and get service principal credentials
temp = !az ad sp create-for-rbac | jq -r '.appId, .password'
sp_client, sp_secret = temp[-2:]

We also generate a json config file for the Python script that creates and submits the AML pipeline. The config file includes AML, Blob storage, Python, and service principal parameters.


In [ ]:
# Create config file for Batch AI job submission script (submit_jobs.py)
submit_jobs_config = {
  "sp_tenant": tenant_id,
  "sp_client": sp_client,
  "sp_secret": sp_secret,
  "resource_group_name": RESOURCE_GROUP,
  "subscription_id": subscription_id,
  "aml_work_space": AML_WORKSPACE,
  "experiment_name": "mm_score",
  "cluster_name": AML_COMPUTE_NAME,
  "location": LOCATION,
  "blob_account": BLOB_ACCOUNT,
  "blob_key": blob_key,
  "models_blob_container": MODELS_CONTAINER,
  "data_blob_container": DATA_CONTAINER,
  "data_blob" : DATA_BLOB,
  "preds_blob_container": PREDS_CONTAINER,
  "pip_packages": PIP_PACKAGES,
  "python_version": PYTHON_VERSION,
  "python_script_name": PYTHON_SCRIPT_NAME,
  "python_script_directory": PYTHON_SCRIPT_DIRECTORY,
  "device_ids": [ 1, 2, 3 ],
  "tags": [ 1, 2, 3, 4, 5 ]
}
with open('sched/bai_pred_config.json', 'w') as f:
    json.dump(submit_jobs_config, f, indent=4)

We will submit AML pipelines on a schedule defined and triggered by a Logic App. The Logic App creates a container instance from ACR and runs a Docker container that executes the pipeline submission. That Docker image can be created and pushed to ACR using the following commands. 

In [ ]:
#Create scheduling docker img
!sudo docker build -f sched/Dockerfile -t {SCHED_DOCKER_IMG} .

In [ ]:
# Create ACR
!az acr create --resource-group {RESOURCE_GROUP} --name {ACR_NAME} --sku Basic
# Enable ACR admin account authentication
!az acr update -n {ACR_NAME} --admin-enabled true
# Get ACR's password 
acr_user = ACR_NAME
acr_password = !az acr credential show --name {ACR_NAME} | jq -r .passwords[0].value
acr_password = acr_password[0]

In [ ]:
# Login to ACR and push docker image
!az acr login --name {ACR_NAME}
!sudo docker push {SCHED_DOCKER_IMG}

Finally, we create the Logic App that acts as a scheduler for this solution. The Logic App and its API connection to Azure Container Instances (ACI) can be created using an Azure Resource Management (ARM) client and corresponding json templates that are stored in the repo.

In [ ]:
# Create credentials and ARM client
credentials = ServicePrincipalCredentials(client_id=sp_client,
                                          secret=sp_secret,
                                          tenant=tenant_id)
arm_client = ResourceManagementClient(credentials, subscription_id)

In [ ]:
# Create an ACI API connection
with open('sched/api_con_template.json') as f:
    aci_api_con_template = json.load(f)

aci_api_con_params = {"location": {"value": LOCATION},
                      "name": {"value": LA_ACI_CON},
                      "subscription_id": {"value": subscription_id}
                      }

aci_api_con_props = {
    'mode': DeploymentMode.incremental,
    'template': aci_api_con_template,
    'parameters': aci_api_con_params
}

arm_client.deployments.create_or_update(RESOURCE_GROUP, LA_ACI_CON, aci_api_con_props)

For the Logic App to run, you would need to authenticate the ACI API connection that has been created.

To do so, navigate to your resource group in the Azure Portal, click on the ACI API connection and authenticate by clicking on the *"This connection is not authenticated"* warning message.

In [ ]:
# Create Logic App
with open('sched/logic_app_template.json') as f:
    logic_app_template = json.load(f)

logic_app_params = {"location": {"value": LOCATION},
                    "resource_group": { "value": RESOURCE_GROUP },
                    "name": {"value": LA_WORKFLOW},
                    "subscription_id": {"value": subscription_id},
                    "container_name": { "value": LA_ACI_CONTAINER_NAME },
                    "container_group": { "value": LA_ACI_CONTAINER_GROUP },
                    "command": { "value": LA_ACI_COMMAND },
                    "image_name": { "value": SCHED_DOCKER_IMG },
                    "acr_pass": { "value": acr_password },
                    "acr_user": { "value": acr_user },
                    "acr_server": { "value": ACR_SERVER },
                    "aci_connection_name": { "value": LA_ACI_CON }
                    }

logic_app_props = {
    'mode': DeploymentMode.incremental,
    'template': logic_app_template,
    'parameters': logic_app_params
}

arm_client.deployments.create_or_update(RESOURCE_GROUP, LA_WORKFLOW, logic_app_props)